# Exercícios - Revisão

É recomendado que cada um reveja os seguintes notebooks:

- [cats-and-dogs-1 Data augmentation](cats-and-dogs-1.ipynb)
- 

# Projeto - Submissão Kaggle: Dogs vs. Cats Redux: Kernels Edition

O projeto nestas duas semanas é fazer a submissão no Kaggle do desafio Dogs vs. Cats.

- Você precisa se registrar no site do Kaggle para obter um username e senha.
- Veja os detalhes da competição neste site: 

  - [https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition)

## Plano de Ação  

- Carregar e preparar os dados para treinamento e validação
- Treinar a rede
- Gerar a predição no conjunto de testes
- Validar as predições feitas
- Submeter as predições

## Formato do arquivo de submissão

O formato do arquivo csv de submissão ao Kaggle tem a seguinte organização:
```
imageId,isDog
1242, .3984
3947, .1000
4539, .9082
2345, .0000
```

O `imageId` deve ser seguido pela probabilidade de ser cachorro.
A métrica utilizada pelo Kaggle é Log Loss.

São 12.500 amostras para teste, sendo que a primeira linha do arquivo .csv contém  o texto `imageId,isDog`


In [11]:
from numpy import genfromtxt
my_data = genfromtxt('../data/sample_submission.csv', delimiter=',')

In [12]:
print(my_data.shape)

(12501, 2)


In [13]:
print(my_data[:5])

[[ nan  nan]
 [ 1.   0.5]
 [ 2.   0.5]
 [ 3.   0.5]
 [ 4.   0.5]]


## Preparando as estruturas dos diretórios

In [1]:
import os, sys
import shutil
import glob
import numpy as np

In [2]:
import keras
from keras.preprocessing import image
from keras import backend as K
from keras.applications.vgg19 import VGG19
K.set_image_data_format("channels_last")

Using TensorFlow backend.


### Copiando os arquivos para meu diretório ../data

Dentro do diretório data iremos copiar o diretório catsdogs_redux e embaixo dele iremos
organizar os diretórios para utilizá-los no treinamento, validação e testes.
A organização dos diretórios segue a utilizada pelo Keras na sua função de geração de dados

In [44]:
Reset = False


# Este comando deve ser executado apenas uma vez

if Reset:
    %rm -rf ../data/catsdogs_redux/
    %cp -r ../../../mylocaldatasets/catsdogs_redux ../data/

### Verificando dataset: 25000 para treino e 12500 para teste

In [45]:
!ls -l ../data/catsdogs_redux/train/ | wc -l 
!ls -l ../data/catsdogs_redux/test/ | wc -l

   25001
   12501


### Separando diretório de validação com 2000 amostras escolhidas aleatoriament do train

In [46]:
# criando diretório de validação
os.makedirs("../data/catsdogs_redux/valid", exist_ok=True) 
g = glob.glob('../data/catsdogs_redux/train/*.jpg')
shuf = np.random.permutation(g)


# ATENÇÃO: A separação no diretório de validação deve ser executada apenas uma vez 
if Reset:
    # Escolhendo apenas as primeiras 2000 amostras embaralhadas e movê-las para valid
    for i in range(2000): 
        os.rename(shuf[i], '../data/catsdogs_redux/valid/' + os.path.basename(shuf[i]))

In [47]:
!ls ../data/catsdogs_redux/valid/ | wc -l 
!ls ../data/catsdogs_redux/train/ | wc -l 

    2000
   23000


### Criando dados para testar inicialmente com poucas amostras

É importante trabalhar com poucos dados durante a fase de depuração do experimento.
Desta forma, o experimento pode ser desenvolvido utilizando um servidor CPU normal.
Apenas depois que tudo está desenvolvido é que se deve utilizar servidor com GPU e
realizar o experimento com treinamento utilizando todos os dados.

- 200 amostras para treinamento
- 50 amostras para validação

In [48]:
# criando diretório de poucas amostras
os.makedirs("../data/catsdogs_redux/sample", exist_ok=True) 
os.makedirs("../data/catsdogs_redux/sample/train", exist_ok=True) 
g = glob.glob('../data/catsdogs_redux/train/*.jpg')
shuf = np.random.permutation(g)

if Reset:
    for i in range(200): 
        shutil.copyfile(shuf[i], '../data/catsdogs_redux/sample/train/' + os.path.basename(shuf[i]))

os.makedirs("../data/catsdogs_redux/sample/valid", exist_ok=True) 
g = glob.glob('../data/catsdogs_redux/train/*.jpg')
shuf = np.random.permutation(g)

if Reset:
    for i in range(50): 
        shutil.copyfile(shuf[i], '../data/catsdogs_redux/sample/valid/' + os.path.basename(shuf[i]))

### Separando cats e dogs em diferentes diretórios (tanto no train como no valid)

In [49]:
# Dividindo no diretório train
os.makedirs('../data/catsdogs_redux/train/cats/', exist_ok=True)
g = glob.glob('../data/catsdogs_redux/train/cat.*.jpg')
for fname in g:
    os.rename(fname,'../data/catsdogs_redux/train/cats/' + os.path.basename(fname) )

os.makedirs('../data/catsdogs_redux/train/dogs/', exist_ok=True)
g = glob.glob('../data/catsdogs_redux/train/dog.*.jpg')
for fname in g:
    os.rename(fname,'../data/catsdogs_redux/train/dogs/' + os.path.basename(fname) )

!ls -l ../data/catsdogs_redux/train/dogs |wc -l
!ls -l ../data/catsdogs_redux/train/cats |wc -l

   11489
   11513


In [50]:
# Dividindo no diretório valid
os.makedirs('../data/catsdogs_redux/valid/cats/', exist_ok=True)
g = glob.glob('../data/catsdogs_redux/valid/cat.*.jpg')
for fname in g:
    os.rename(fname,'../data/catsdogs_redux/valid/cats/' + os.path.basename(fname) )

os.makedirs('../data/catsdogs_redux/valid/dogs/', exist_ok=True)
g = glob.glob('../data/catsdogs_redux/valid/dog.*.jpg')
for fname in g:
    os.rename(fname,'../data/catsdogs_redux/valid/dogs/' + os.path.basename(fname) )

!ls -l ../data/catsdogs_redux/valid/dogs |wc -l
!ls -l ../data/catsdogs_redux/valid/cats |wc -l

    1013
     989


In [51]:
# Dividindo no diretório sample/train
os.makedirs('../data/catsdogs_redux/sample/train/cats/', exist_ok=True)
g = glob.glob('../data/catsdogs_redux/sample/train/cat.*.jpg')
for fname in g:
    os.rename(fname,'../data/catsdogs_redux/sample/train/cats/' + os.path.basename(fname) )

os.makedirs('../data/catsdogs_redux/sample/train/dogs/', exist_ok=True)
g = glob.glob('../data/catsdogs_redux/sample/train/dog.*.jpg')
for fname in g:
    os.rename(fname,'../data/catsdogs_redux/sample/train/dogs/' + os.path.basename(fname) )

!ls -l ../data/catsdogs_redux/sample/train/dogs |wc -l
!ls -l ../data/catsdogs_redux/sample/train/cats |wc -l

     110
      92


In [52]:
# Dividindo no diretório sample/valid
os.makedirs('../data/catsdogs_redux/sample/valid/cats/', exist_ok=True)
g = glob.glob('../data/catsdogs_redux/sample/valid/cat.*.jpg')
for fname in g:
    os.rename(fname,'../data/catsdogs_redux/sample/valid/cats/' + os.path.basename(fname) )

os.makedirs('../data/catsdogs_redux/sample/valid/dogs/', exist_ok=True)
g = glob.glob('../data/catsdogs_redux/sample/valid/dog.*.jpg')
for fname in g:
    os.rename(fname,'../data/catsdogs_redux/sample/valid/dogs/' + os.path.basename(fname) )

!ls -l ../data/catsdogs_redux/sample/valid/dogs |wc -l
!ls -l ../data/catsdogs_redux/sample/valid/cats |wc -l

      29
      23


In [53]:
os.makedirs('../data/catsdogs_redux/results')
os.makedirs('../data/catsdogs_redux/sample/results')

## Criando a Rede e Treinando

In [54]:
batch_size=64
epochs=3

#Configure path para sample/ e depois que estiver tudo testado, tirar
path = '../data/catsdogs_redux/sample/'
#path = '../data/catsdogs_redux/'

test_path  = path + 'test/'
train_path = path + 'train/'
valid_path = path + 'valid/'
results_path= path + 'results/'

In [ ]:
model = VGG19(weights='imagenet',include_top=False)

In [60]:
gen=image.ImageDataGenerator()

batches = gen.flow_from_directory(train_path, 
                                  target_size=(224,224),
                                  class_mode='categorical', 
                                  shuffle=True, 
                                  batch_size=batch_size)
val_batches = gen.flow_from_directory(valid_path,
                                     target_size=(224,224),
                                     class_mode='categorical',
                                     shuffle=True,
                                     batch_size=batch_size)

Found 200 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
